In [ ]:
!pip install dateparser
!pip install datefinder

# Originaly the data was saved in .txt file, so I had to do some processing
### In this notebook the test data for NER models is processed into structured dict and saved from .txt file to .json
### Function convert_to_standard_date can also be found in scripts.py

In [1]:
with open('test_data.txt', mode='r') as f:
    data = f.read()

In [2]:
def extract_requests_and_data(text):
    requests = []
    data = []

    def find_next(text):
        start_quote = text.find('Request: "') + len('Request:"')
        end_quote = text.find('"', start_quote + 1)

        if start_quote != -1 and end_quote != -1:
            requests.append(text[start_quote + 1:end_quote])
            remaining_text = text[end_quote + 1:]
            find_data(remaining_text)
        else:
            return

    def find_data(text):
        data_start = text.find('Data:')
        if data_start != -1:
            end_of_data = text.find('\n', data_start)
            data.append(text[data_start + 6:end_of_data])
            remaining_text = text[end_of_data + 1:]
            find_next(remaining_text)
        else:
            return

    find_next(text)
    return requests, data


requests_extracted, data_extracted = extract_requests_and_data(data)


In [3]:
# Add structure to the data
dict_of_data = {'request': [],
                'data': []                   
               }

import dateparser
# Convert date to standard format
def convert_to_standard_date(date_string):
    parsed_date = dateparser.parse(date_string)
    if parsed_date:
        return parsed_date.strftime('%d-%m-%Y')
    else:
        return "Unspecified"


for idx, data in enumerate(data_extracted):
    dict_of_data['request'].append(requests_extracted[idx])

    name, departure, destination, date = data.split(', ')

    date = convert_to_standard_date(date)
    dict_of_data['data'].append({
        'name': name,
        'departure': departure,
        'destination': destination,
        'date': date
    })
    

In [4]:
import json
with open('test_data.json', 'w') as f:
    json.dump(dict_of_data, f)

# Test

In [5]:
with open('test_data.json') as json_file:
    loaded_dict = json.load(json_file)

In [6]:
len(loaded_dict['request']), len(loaded_dict['data'])

(120, 120)

In [7]:
loaded_dict['request'][84], loaded_dict['data'][84]

('Good day, my name is Victoria Thompson. I am considering a flight from Paris to Dubai around the 28th of February 2024. Could you kindly provide information on flight options for this date?',
 {'name': 'Victoria Thompson',
  'departure': 'Paris',
  'destination': 'Dubai',
  'date': '28-02-2024'})

In [8]:
def load_test_data(file_name='test_data.json'):
    with open(file_name) as json_file:
        loaded_dict = json.load(json_file)

    user_requests = loaded_dict['request']
    required_data = loaded_dict['data']

    return user_requests, required_data

user_requests, required_data = load_test_data()

In [9]:
user_requests[84], required_data[84]

('Good day, my name is Victoria Thompson. I am considering a flight from Paris to Dubai around the 28th of February 2024. Could you kindly provide information on flight options for this date?',
 {'name': 'Victoria Thompson',
  'departure': 'Paris',
  'destination': 'Dubai',
  'date': '28-02-2024'})